# Computação Bayesiana Aproximada

Pense Bayes, Segunda Edição

Copyright 2020 Allen B. Downey

Licença: [Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)](https://creativecommons.org/licenses/by-nc-sa/4.0/)

In [1]:
# If we're running on Colab, install libraries

import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install empiricaldist

In [2]:
# Get utils.py

from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)
    
download('https://github.com/AllenDowney/ThinkBayes2/raw/master/soln/utils.py')

In [3]:
from utils import set_pyplot_params
set_pyplot_params()

Este capítulo introduz um método de último recurso para os problemas mais complexos, a Computação Bayesiana Aproximada (ABC).

Digo que é um último recurso porque normalmente requer mais cálculos do que outros métodos, por isso, se pode resolver um problema de qualquer outra forma, deve fazê-lo.

Contudo, para os exemplos deste capítulo, o ABC não é apenas fácil de implementar; é também eficiente.

O primeiro exemplo é a minha solução para um problema colocado por um paciente

com um tumor renal.

Utilizo dados de uma revista médica para modelar o crescimento tumoral, e utilizo simulações para estimar a idade de um tumor com base no seu tamanho.

O segundo exemplo é um modelo de contagem de células, que tem aplicações em biologia, medicina, e zimurgia (fabrico de cerveja).

Dada uma contagem de células de uma amostra diluída, estimamos a concentração de células.

Finalmente, como um exercício, terá a oportunidade de trabalhar num divertido problema de contagem de meias.



## O problema do tumor nos rins

Sou um leitor frequente e contribuinte ocasional para a Internet

fórum de estatísticas em <http://reddit.com/r/statistics>. 

Em Novembro de 2011, li a seguinte mensagem:

> "Tenho cancro do rim em fase IV e estou a tentar determinar se o cancro se formou antes de me reformar dos militares. ... Dadas as datas de reforma e detecção é possível determinar quando houve uma probabilidade de 50/50 de eu ter desenvolvido a doença? É possível determinar a probabilidade na data da reforma? O meu tumor era de 15,5 cm x 15 cm no momento da detecção. Grau II".

Contactei o autor da mensagem para obter mais informações; I

Aprendeu que os veteranos obtêm benefícios diferentes se for "mais provável do que não" que um tumor se tenha formado enquanto estavam no serviço militar (entre outras considerações).

Por isso, concordo em ajudá-lo a responder à sua pergunta.

Como os tumores renais crescem lentamente, e muitas vezes não causam sintomas, são por vezes deixados sem tratamento. Como resultado, os médicos podem observar a taxa de crescimento de tumores não tratados, comparando exames do mesmo paciente em momentos diferentes. Vários artigos relataram estas taxas de crescimento.

Para a minha análise utilizei dados de um artigo por [Zhang et al](https://pubs.rsna.org/doi/full/10.1148/radiol.2501071712). 

Relatam as taxas de crescimento de duas formas:

* Tempo de duplicação volumétrica, que é o tempo que levaria para que um tumor duplicasse de tamanho.

* Tempo de duplicação recíproca (RDT), que é o número de duplicações por ano.

A secção seguinte mostra como trabalhamos com estas taxas de crescimento.

Zhang et al, Distribuição das Taxas de Crescimento de Tumor Renal Determinadas

    usando Medições de CT Volumétricas em Série, Janeiro de 2009

    *Radiologia*, 250, 137-144.


https://pubs.rsna.org/doi/full/10.1148/radiol.2501071712

## Um modelo de crescimento simples

Vamos começar com um modelo simples de crescimento tumoral baseado em dois pressupostos:

* Os tumores crescem com um tempo de duplicação constante, e 

* Têm uma forma aproximadamente esférica.

E vou definir dois pontos no tempo:

* 't1' é quando o meu correspondente se reformou.

* O `t2` é quando o tumor foi detectado.

O tempo entre o `t1' e o `t2' foi de cerca de 9,0 anos.

Como exemplo, vamos assumir que o diâmetro do tumor era de 1 cm em `t1`, e estimar o seu tamanho em `t2`.

Vou utilizar a seguinte função para calcular o volume de uma esfera com um determinado diâmetro.

In [4]:
import numpy as np

def calc_volume(diameter):
    """Converts a diameter to a volume."""
    factor = 4 * np.pi / 3
    return factor * (diameter/2.0)**3

Assumindo que o tumor é esférico, podemos calcular o seu volume em `t1`.

In [5]:
d1 = 1
v1 = calc_volume(d1)
v1

O tempo médio de duplicação do volume relatado por Zhang et al. é de 811 dias, o que corresponde a uma RDT de 0,45 duplicações por ano.

In [6]:
median_doubling_time = 811
rdt = 365 / median_doubling_time
rdt

Podemos calcular o número de duplicações que teriam acontecido no intervalo entre `t1` e `t2`:

In [7]:
interval = 9.0
doublings = interval * rdt
doublings

Dado o `v1' e o número de duplicações, podemos calcular o volume em `t2'.

In [8]:
v2 = v1 * 2**doublings
v2

A função seguinte calcula o diâmetro de uma esfera com o volume dado.

In [9]:
def calc_diameter(volume):
    """Converts a volume to a diameter."""
    factor = 3 / np.pi / 4
    return 2 * (factor * volume)**(1/3)

Assim, podemos calcular o diâmetro do tumor em `t2`:

In [10]:
d2 = calc_diameter(v2)
d2

Se o diâmetro do tumor fosse de 1 cm a `t1`, e crescesse à taxa mediana, o diâmetro seria de cerca de 2,5 cm a `t2`.

Este exemplo demonstra o modelo de crescimento, mas não responde à pergunta que o meu correspondente fez.

## Um modelo mais geral

Dado o tamanho de um tumor no momento do diagnóstico, gostaríamos de saber a distribuição da sua idade.

Para o encontrar, faremos simulações de crescimento tumoral para obter a distribuição do tamanho condicionado à idade. 

Depois calcularemos a distribuição da idade condicionada ao tamanho.

A simulação começa com um pequeno tumor e executa estes passos:

1.  Escolha um valor a partir da distribuição das taxas de crescimento.

2.  Calcular o tamanho do tumor no fim de um intervalo.

3.  Repetir até que o tumor exceda o tamanho máximo relevante.

Portanto, a primeira coisa de que precisamos é da distribuição das taxas de crescimento.

Utilizando os números no artigo de Zhange et al., criei uma matriz, `rdt_sample`, que contém valores estimados de RDT para os 53 pacientes do estudo.

Mais uma vez, RDT significa "tempo de duplicação recíproca", que está em duplicações por ano.

Assim, se `rdt=1`, um tumor duplicaria de volume em um ano.

Se `rdt=2`, duplicaria duas vezes; ou seja, o volume quadruplicaria.

E, se `rdt=-1`, reduziria para metade o seu volume.

In [11]:
# Data from the histogram in Figure 3

import numpy as np
from empiricaldist import Pmf

counts = [2, 29, 11, 6, 3, 1, 1]
rdts = np.arange(-1, 6) + 0.01
pmf_rdt = Pmf(counts, rdts)
pmf_rdt.normalize()

In [12]:
# Data from the scatter plot in Figure 4

rdts = [5.089,  3.572,  3.242,  2.642,  1.982,  1.847,  1.908,  1.798,
        1.798,  1.761,  2.703, -0.416,  0.024,  0.869,  0.746,  0.257,
        0.269,  0.086,  0.086,  1.321,  1.052,  1.076,  0.758,  0.587,
        0.367,  0.416,  0.073,  0.538,  0.281,  0.122, -0.869, -1.431,
        0.012,  0.037, -0.135,  0.122,  0.208,  0.245,  0.404,  0.648,
        0.673,  0.673,  0.563,  0.391,  0.049,  0.538,  0.514,  0.404,
        0.404,  0.33,  -0.061,  0.538,  0.306]

rdt_sample = np.array(rdts)
len(rdt_sample)

Podemos utilizar a amostra de RDTs para estimar o PDF da distribuição.

In [13]:
from utils import kde_from_sample

qs = np.linspace(-2, 6, num=201)
pmf_rdt = kde_from_sample(rdt_sample, qs)

In [14]:
1 / pmf_rdt.median() * 365

Aqui está o que parece.

In [15]:
from utils import decorate

pmf_rdt.plot(label='rdts')

decorate(xlabel='Reciprocal doubling time (RDT)',
         ylabel='PDF',
         title='Distribution of growth rates')

Na secção seguinte utilizaremos esta distribuição para simular o crescimento tumoral. 

## Simulação

Agora estamos prontos para executar as simulações.

Começando com um pequeno tumor, vamos simular uma série de intervalos até que o tumor atinja um tamanho máximo.

No início de cada intervalo simulado, vamos escolher um valor da distribuição das taxas de crescimento e calcular o tamanho do tumor no final.

Escolhi um intervalo de 245 dias (cerca de 8 meses) porque esse é o

mediana do tempo entre medições na fonte de dados

Para o diâmetro inicial escolhi 0,3 cm, porque os carcinomas mais pequenos do que isso são menos susceptíveis de serem invasivos e menos susceptíveis de terem o fornecimento de sangue necessário para um crescimento rápido (ver [this page on carcinoma](http://en.wikipedia.org/wiki/Carcinoma_in_situ)).

Para o diâmetro máximo, escolhi 20 cm. 

In [16]:
interval = 245 / 365      # year
min_diameter = 0.3        # cm
max_diameter = 20         # cm

Vou utilizar o `calc_volume` para calcular os volumes iniciais e máximos:

In [17]:
v0 = calc_volume(min_diameter)
vmax = calc_volume(max_diameter)
v0, vmax

A seguinte função executa a simulação.

In [18]:
import pandas as pd

def simulate_growth(pmf_rdt):
    """Simulate the growth of a tumor."""
    age = 0
    volume = v0
    res = []
    
    while True:
        res.append((age, volume))
        if volume > vmax:
            break

        rdt = pmf_rdt.choice()
        age += interval 
        doublings = rdt * interval
        volume *= 2**doublings
        
    columns = ['age', 'volume']
    sim = pd.DataFrame(res, columns=columns)
    sim['diameter'] = calc_diameter(sim['volume'])
    return sim

O `simulate_growth` toma como parâmetro um `Pmf` que representa a distribuição de RDT.

Inicializa a idade e o volume do tumor, depois executa um loop que simula um intervalo de cada vez.

De cada vez que passa pelo laço, verifica o volume do tumor e sai se este exceder o `vmax`.

Caso contrário, escolhe um valor de `pmf_rdt` e actualiza `age` e `volume`.  Uma vez que a `rdt` está em duplicações por ano, multiplicamos por `intervalo` para calcular o número de duplicações durante cada intervalo.

No final do laço, `simulate_growth` coloca os resultados num `DataFrame` e calcula o diâmetro que corresponde a cada volume.

Eis como chamamos a esta função:

In [19]:
np.random.seed(17)

In [20]:
sim = simulate_growth(pmf_rdt)

Aqui estão os resultados para os primeiros intervalos:

In [21]:
sim.head(3)

E os últimos intervalos.

In [22]:
sim.tail(3)

Para mostrar os resultados graficamente, vou fazer 101 simulações:

In [23]:
np.random.seed(17)

In [24]:
sims = [simulate_growth(pmf_rdt) for _ in range(101)]

E traçar os resultados.

In [25]:
import matplotlib.pyplot as plt

diameters = [4, 8, 16]
for diameter in diameters:
    plt.axhline(diameter,
                color='C5', linewidth=2, linestyle='dotted')

for sim in sims:
    plt.plot(sim['age'], sim['diameter'],
             color='C1', linewidth=0.5, alpha=0.5)
    
decorate(xlabel='Tumor age (years)',
         ylabel='Diameter (cm, log scale)',
         ylim=[0.2, 20],
         yscale='log')

yticks = [0.2, 0.5, 1, 2, 5, 10, 20]
plt.yticks(yticks, yticks);

Nesta figura, cada linha fina e sólida mostra o crescimento simulado de um tumor ao longo do tempo, com diâmetro numa escala de tronco.

As linhas pontilhadas estão a 4, 8, e 16 cm.

Ao ler através das linhas pontilhadas, pode obter uma noção da distribuição da idade em cada tamanho.

Por exemplo, lendo através da linha superior, vemos que a idade de um tumor de 16 cm pode ser tão baixa 10 anos ou tão alta como 40 anos, mas é mais provável que seja entre os 15 e 30 anos.

Para calcular esta distribuição mais precisamente, podemos interpolar as curvas de crescimento para ver quando cada uma delas passa por um determinado tamanho.

A função seguinte toma os resultados das simulações e devolve a idade em que cada tumor atingiu um determinado diâmetro.

In [26]:
from scipy.interpolate import interp1d

def interpolate_ages(sims, diameter):
    """Estimate the age when each tumor reached a given size."""
    ages = []
    for sim in sims:
        interp = interp1d(sim['diameter'], sim['age'])
        age = interp(diameter)
        ages.append(float(age))
    return ages

Podemos chamar a esta função desta forma:

In [27]:
from empiricaldist import Cdf

ages = interpolate_ages(sims, 15)
cdf = Cdf.from_seq(ages)
print(cdf.median(), cdf.credible_interval(0.9))

Para um tumor com 15 cm de diâmetro, a idade média é de cerca de 22 anos, o intervalo credível de 90% situa-se entre 13 e 34 anos, e a probabilidade de se ter formado há menos de 9 anos é inferior a 1%.

In [28]:
1 - cdf(9.0)

Mas este resultado baseia-se em duas decisões de modelação que são potencialmente problemáticas:

* Nas simulações, a taxa de crescimento durante cada intervalo é independente das taxas de crescimento anteriores. Na realidade, é plausível que os tumores que cresceram rapidamente no passado sejam passíveis de crescer rapidamente no futuro. Por outras palavras, existe provavelmente uma correlação em série na taxa de crescimento.

* Para converter de medida linear para volume, assumimos que os tumores são aproximadamente esféricos.

Em experiências adicionais, implementei uma simulação que escolhe taxas de crescimento com correlação em série; o efeito é que os tumores de crescimento rápido crescem mais rapidamente e os tumores de crescimento lento crescem mais lentamente.

No entanto, com uma correlação moderada (0,5), a probabilidade de um tumor de 15 cm ter menos de 9 anos de idade é apenas de cerca de 1%. 

A suposição de que os tumores são esféricos é provavelmente boa para tumores até alguns centímetros, mas não para um tumor com dimensões lineares de 15,5 x 15 cm.

Se, como parece provável, um tumor deste tamanho for relativamente plano, poderá ter o mesmo volume que uma esfera de 6 cm.

Mas mesmo com este menor volume e correlação 0,5, a probabilidade deste tumor ter menos de 9 anos é de cerca de 5%.

Assim, mesmo tendo em conta erros de modelação, é improvável que um tumor tão grande possa ter-se formado depois de o meu correspondente se ter reformado do serviço militar.

A figura seguinte mostra a distribuição das idades para tumores com diâmetros de 4, 8, e 15 cm.

In [29]:
for diameter in diameters:
    ages = interpolate_ages(sims, diameter)
    cdf = Cdf.from_seq(ages)
    cdf.plot(label=f'{diameter} cm')
    
decorate(xlabel='Tumor age (years)',
         ylabel='CDF')

## Cálculo Bayesiano aproximado

Neste momento, poderá perguntar-se porque é que este exemplo está num livro sobre estatísticas Bayesianas.

Nunca definimos uma distribuição prévia ou fizemos uma actualização Bayesiana.

Porque não? Porque não tínhamos de o fazer.

Em vez disso, utilizámos simulações para calcular idades e tamanhos para uma colecção de hipotéticos tumores.

Depois, implicitamente, utilizámos os resultados da simulação para formar uma distribuição conjunta de idade e tamanho.

Se seleccionarmos uma coluna da distribuição conjunta, obtemos uma distribuição de tamanho condicionada à idade.

Se seleccionarmos uma fila, obtemos uma distribuição de idade condicionada ao tamanho.

Portanto, este exemplo é como os que vimos em <<_Probabilidade>>: se tiver todos os dados, não precisa do teorema de Bayes; pode calcular as probabilidades contando.

Este exemplo é um primeiro passo em direcção à Computação Bayesiana Aproximada (ABC).

O exemplo seguinte é um segundo passo.

## Células de contagem

Este exemplo vem de [this blog post](https://dataorigami.net/blogs/napkin-folding/bayesian-cell-counting), de Cameron Davidson-Pilon.

Nele, ele modela o processo que os biólogos utilizam para estimar a concentração de células numa amostra de líquido.

O exemplo que apresenta é a contagem de células numa "pasta de levedura", que é uma mistura de levedura e água utilizada na fabricação de cerveja.

Há duas etapas no processo:

* Primeiro, o chorume é diluído até que a concentração seja suficientemente baixa para que seja prático contar células.

* Depois é colocada uma pequena amostra num hemocitómetro, que é uma lâmina de microscópio especializada que contém uma quantidade fixa de líquido numa grelha rectangular.

As células e a grelha são visíveis num microscópio, tornando possível contar as células com precisão.

Como exemplo, suponhamos que começamos com uma lama de levedura com concentração desconhecida de células.

Começando com uma amostra de 1 mL, diluímo-la, adicionando-a a um agitador com 9 mL de água e misturando bem.

Depois diluímo-lo novamente, e depois uma terceira vez.

Cada diluição reduz a concentração num factor de 10, pelo que três diluições reduzem a concentração num factor de 1000.

Depois adicionamos a amostra diluída ao hemocitómetro, que tem uma capacidade de 0,0001 mL espalhada por uma grelha de 5x5.



Este processo é suficientemente simples, mas em todas as fases existem fontes de erro:

* Durante o processo de diluição, os líquidos são medidos utilizando pipetas que introduzem erros de medição.

* A quantidade de líquido no hemocitómetro pode variar em relação à especificação.

* Durante cada etapa do processo de amostragem, podemos seleccionar mais ou menos do que o número médio de células, devido à variação aleatória.

Davidson-Pilon apresenta um modelo PyMC que descreve estes erros.

Começarei por replicar o seu modelo; depois adaptá-lo-emos para o ABC.

Suponhamos que existem 25 quadrados na grelha, contamos 5 deles, e o número total de células é 49.

In [30]:
total_squares = 25
squares_counted = 5
yeast_counted = 49

Eis a primeira parte do modelo, que define a distribuição prévia de `yeast_conc`, que é a concentração de levedura que estamos a tentar estimar.

`shaker1_vol` é o volume real de água no primeiro agitador, que deve ser de 9 mL, mas pode ser superior ou inferior, com desvio padrão de 0,05 mL.

`shaker2_vol` e `shaker3_vol` são os volumes do segundo e terceiro agitadores.

In [31]:
import pymc3 as pm
billion = 1e9

with pm.Model() as model:
    yeast_conc = pm.Normal("yeast conc", 
                           mu=2 * billion, sd=0.4 * billion)

    shaker1_vol = pm.Normal("shaker1 vol", 
                               mu=9.0, sd=0.05)
    shaker2_vol = pm.Normal("shaker2 vol", 
                               mu=9.0, sd=0.05)
    shaker3_vol = pm.Normal("shaker3 vol", 
                               mu=9.0, sd=0.05)

Agora, a amostra retirada da lama de levedura é supostamente de 1 mL, mas pode ser mais ou menos.

E de forma semelhante para a amostra do primeiro agitador e do segundo agitador.

As seguintes variáveis modelam estes passos.

In [32]:
with model:
    yeast_slurry_vol = pm.Normal("yeast slurry vol",
                                    mu=1.0, sd=0.01)
    shaker1_to_shaker2_vol = pm.Normal("shaker1 to shaker2",
                                    mu=1.0, sd=0.01)
    shaker2_to_shaker3_vol = pm.Normal("shaker2 to shaker3",
                                    mu=1.0, sd=0.01)

Dados os volumes reais nas amostras e nos agitadores, podemos calcular a diluição efectiva, "diluição_final", que deve ser de 1000, mas pode ser superior ou inferior.

In [33]:
with model:
    dilution_shaker1 = (yeast_slurry_vol / 
                        (yeast_slurry_vol + shaker1_vol))
    dilution_shaker2 = (shaker1_to_shaker2_vol / 
                        (shaker1_to_shaker2_vol + shaker2_vol))
    dilution_shaker3 = (shaker2_to_shaker3_vol / 
                        (shaker2_to_shaker3_vol + shaker3_vol))
    
    final_dilution = (dilution_shaker1 * 
                      dilution_shaker2 * 
                      dilution_shaker3)

O passo seguinte é colocar uma amostra do terceiro agitador na câmara do hemocitador.

A capacidade da câmara deve ser de 0,0001 mL, mas pode variar; para descrever esta variação, utilizaremos uma distribuição gama, o que garante que não geramos valores negativos.

In [34]:
with model:
    chamber_vol = pm.Gamma("chamber_vol", 
                           mu=0.0001, sd=0.0001 / 20)

Em média, o número de células na câmara é o produto da concentração real, da diluição final, e do volume da câmara.

Mas o número real pode variar; usaremos uma distribuição de Poisson para modelar esta variação.

In [35]:
with model:
    yeast_in_chamber = pm.Poisson("yeast in chamber", 
        mu=yeast_conc * final_dilution * chamber_vol)

Finalmente, cada célula da câmara estará num dos quadrados que contamos com probabilidade `p=quadrados_contados/total_quadrados`.

Assim, a contagem real segue uma distribuição binomial.

In [36]:
with model:
    count = pm.Binomial("count", 
                        n=yeast_in_chamber, 
                        p=squares_counted/total_squares,
                        observed=yeast_counted)

Com o modelo especificado, podemos utilizar a "amostra" para gerar uma amostra a partir da distribuição posterior.

In [37]:
options = dict(return_inferencedata=False)

with model:
    trace = pm.sample(1000, **options)

E podemos utilizar a amostra para estimar a distribuição posterior de `yeast_conc` e calcular estatísticas sumárias.

In [38]:
posterior_sample = trace['yeast conc'] / billion
cdf_pymc = Cdf.from_seq(posterior_sample)
print(cdf_pymc.mean(), cdf_pymc.credible_interval(0.9))

A média posterior é de cerca de 2,3 mil milhões de células por mL, com um intervalo credível de 90% de 1,8 e 2,7.

Até agora temos seguido as pegadas de Davidson-Pilon.


Mas também oferece uma oportunidade para demonstrar o ABC.

## Contagem de células com ABC

A ideia fundamental do ABC é que utilizamos a distribuição prévia para gerar uma amostra dos parâmetros, e depois simular o sistema para cada conjunto de parâmetros na amostra.

Neste caso, uma vez que já temos um modelo PyMC, podemos utilizar `sample_prior_predictive` para fazer a amostragem e a simulação.

In [39]:
with model:
    prior_sample = pm.sample_prior_predictive(10000)

O resultado é um dicionário que contém amostras da distribuição prévia dos parâmetros e da distribuição preditiva prévia da "contagem".

In [40]:
count = prior_sample['count']
print(count.mean())

Agora, para gerar uma amostra a partir da distribuição posterior, seleccionaremos apenas os elementos da amostra anterior onde a saída da simulação, "contagem", corresponde aos dados observados, 49.

In [41]:
mask = (count == 49)
mask.sum()

Podemos utilizar `mask` para seleccionar os valores de `yeast_conc` para as simulações que produzem os dados observados.

In [42]:
posterior_sample2 = prior_sample['yeast conc'][mask] / billion

E podemos utilizar a amostra posterior para estimar o CDF da distribuição posterior.

In [43]:
cdf_abc = Cdf.from_seq(posterior_sample2)
print(cdf_abc.mean(), cdf_abc.credible_interval(0.9))

A média posterior e o intervalo credível são semelhantes ao que temos com o MCMC.

Eis como são as distribuições.

In [44]:
cdf_pymc.plot(label='MCMC', style=':')
cdf_abc.plot(label='ABC')

decorate(xlabel='Yeast concentration (cells/mL)',
         ylabel='CDF',
         title='Posterior distribution',
         xlim=(1.4, 3.4))

As distribuições são semelhantes, mas os resultados do ABC são mais ruidosos porque o tamanho da amostra é menor.

## Quando chegamos à parte aproximada?

Os exemplos até agora são semelhantes aos da Computação Bayesiana Aproximada, mas nenhum dos dois demonstra todos os elementos do ABC.

Mais genericamente, o ABC é caracterizado por:

1. Uma distribuição prévia de parâmetros.

2. Uma simulação do sistema que gera os dados.

3. Um critério para quando devemos aceitar que a saída da simulação corresponda aos dados.

O exemplo do tumor renal era atípico porque não representava explicitamente a distribuição prévia da idade.

Como as simulações geram uma distribuição conjunta de idade e tamanho, conseguimos obter a distribuição posterior marginal de idade directamente a partir dos resultados.

O exemplo da levedura é mais típico porque representamos explicitamente a distribuição dos parâmetros.

Mas aceitámos apenas simulações em que a saída corresponda exactamente aos dados.

O resultado é aproximado no sentido em que temos uma amostra da distribuição posterior e não da distribuição posterior em si.

Mas não é aproximada no sentido de Computação Bayesiana Aproximada, que normalmente aceita simulações em que a saída corresponde apenas aproximadamente aos dados.

Para mostrar como isso funciona, vou alargar o exemplo da levedura com um critério de correspondência aproximado.

Na secção anterior, aceitámos uma simulação se a saída for precisamente 49 e rejeitámo-la de outra forma.

Como resultado, obtivemos apenas algumas centenas de amostras em 10.000 simulações, o que não é muito eficiente.

Podemos fazer melhor uso das simulações se dermos "crédito parcial" quando a produção estiver perto de 49.

Mas quão perto?  E quanto crédito?

Uma forma de responder que é apoiar até ao penúltimo passo da simulação, onde sabemos o número de células na câmara, e usamos a distribuição binomial para gerar a contagem final.

Se houver células "não" na câmara, cada uma tem uma probabilidade "p" de ser contada, dependendo se cai num dos quadrados da grelha que são contados.

Podemos extrair `n` da amostra anterior, desta forma:

In [45]:
n = prior_sample['yeast in chamber']
n.shape

E calcular `p` desta forma:

In [46]:
p = squares_counted/total_squares
p

Agora a ideia é esta: utilizaremos a distribuição binomial para calcular a probabilidade dos dados, "contados_ao_mês", para cada valor de "n" e o valor fixo de "p".

In [47]:
from scipy.stats import binom

likelihood = binom(n, p).pmf(yeast_counted).flatten()

In [48]:
likelihood.shape

Quando a contagem esperada, `n * p`, está próxima da contagem real, a `probabilidade' é relativamente alta; quando está mais longe, a `probabilidade' é mais baixa.

O seguinte é um gráfico de dispersão destas probabilidades versus as contagens esperadas.

In [49]:
plt.plot(n*p, likelihood, '.', alpha=0.03, color='C2')

decorate(xlabel='Expected count (number of cells)',
         ylabel='Likelihood')

Não podemos utilizar estas probabilidades para fazer uma actualização Bayesiana porque estão incompletas; ou seja, cada probabilidade é a probabilidade dos dados dados dados dados "n", que é o resultado de uma única simulação.

Mas nós *podemos* utilizá-los para ponderar os resultados das simulações.

Em vez de exigirmos que a saída da simulação corresponda exactamente aos dados, utilizaremos a probabilidade de dar crédito parcial quando a saída estiver próxima.

Eis como: Vou construir um `Pmf` que contém concentrações de levedura como quantidades e as probabilidades como probabilidades não normalizadas.

In [50]:
qs = prior_sample['yeast conc'] / billion
ps = likelihood
posterior_pmf = Pmf(ps, qs)

Neste `Pmf`, os valores de `yeast_conc` que produzem resultados próximos do mapa de dados para probabilidades mais elevadas.

Se ordenarmos as quantidades e normalizarmos as probabilidades, o resultado é uma estimativa da distribuição posterior.

In [51]:
posterior_pmf.sort_index(inplace=True)
posterior_pmf.normalize()

print(posterior_pmf.mean(), posterior_pmf.credible_interval(0.9)) 

A média posterior e o intervalo credível são semelhantes aos valores que obtivemos da MCMC.

E aqui está como são as distribuições posteriores.

In [52]:
cdf_pymc.plot(label='MCMC', style=':')
#cdf_abc.plot(label='ABC')
posterior_pmf.make_cdf().plot(label='ABC2')

decorate(xlabel='Yeast concentration (cells/mL)',
         ylabel='CDF',
         title='Posterior distribution',
         xlim=(1.4, 3.4))

As distribuições são semelhantes, mas os resultados da MCMC são um pouco mais ruidosos.

Neste exemplo, o ABC é mais eficiente do que o MCMC, exigindo menos cálculos para gerar uma melhor estimativa da distribuição posterior.

Mas isso é invulgar; normalmente o ABC requer muitos cálculos.

Por essa razão, é geralmente um método de último recurso.

## Resumo

Neste capítulo, vimos dois exemplos de Computação Bayesiana Aproximada (ABC), baseada em simulações de crescimento tumoral e contagem de células.

Os elementos definitivos do ABC são:

1. Uma distribuição prévia de parâmetros.

2. Uma simulação do sistema que gera os dados.

3. Um critério para quando devemos aceitar que a saída da simulação corresponda aos dados.

O ABC é particularmente útil quando o sistema é demasiado complexo para ser modelado com ferramentas como o PyMC.

Por exemplo, pode envolver uma simulação física baseada em equações diferenciais.

Nesse caso, cada simulação pode requerer cálculos substanciais, e muitas simulações podem ser necessárias para estimar a distribuição posterior.

A seguir, terá a oportunidade de praticar com mais um exemplo.

## Exercícios

**Exercício:** Este exercício baseia-se em [a blog post by Rasmus BÃ¥Ã¥th](http://www.sumsar.net/blog/2014/10/tiny-data-and-the-socks-of-karl-broman), motivado por um tweet de Karl Broman, que escreveu:

> Que as primeiras 11 meias na lavandaria são distintas sugere que existem muitas meias.

Suponha que tira 11 meias da lavandaria e descobre que nenhuma das duas faz um par combinado.  Faça uma estimativa do número de meias na lavandaria.

Para resolver este problema, utilizaremos o modelo BÃ¥¥¥th sugerido, que se baseia nestes pressupostos:

* A roupa contém algum número de pares de meias, `n_pairs', mais algum número de meias ímpares (não pareadas), `n_odds'.

* Os pares de meias são diferentes um do outro e diferentes das meias não emparelhadas; por outras palavras, o número de meias de cada tipo é de 1 ou 2, nunca mais.

Vamos utilizar as distribuições anteriores BÃ¥¥¥th sugeridas, que são

* O número de meias segue uma distribuição binomial negativa com média 30 e desvio padrão 15.

* A proporção de meias que são pareadas segue uma distribuição beta com os parâmetros `alpha=15` e `beta=2`.

No caderno deste capítulo, vou definir estes antecedentes.  Depois poderá simular o processo de amostragem e utilizar o ABC para estimar as distribuições posteriores.

Para começar, vou definir os antecedentes.

In [53]:
from scipy.stats import nbinom, beta

mu = 30
p = 0.8666666
r = mu * (1-p) / p

prior_n_socks = nbinom(r, 1-p)
prior_n_socks.mean(), prior_n_socks.std()

In [54]:
prior_prop_pair = beta(15, 2)
prior_prop_pair.mean()

In [55]:
qs = np.arange(90)
ps = prior_n_socks.pmf(qs)
pmf = Pmf(ps, qs)
pmf.normalize()

pmf.plot(label='prior', drawstyle='steps')

decorate(xlabel='Number of socks',
         ylabel='PMF')

In [56]:
from utils import pmf_from_dist

qs = np.linspace(0, 1, 101)
pmf = pmf_from_dist(prior_prop_pair, qs)
pmf.plot(label='prior', color='C1')

decorate(xlabel='Proportion of socks in pairs',
         ylabel='PDF')

Podemos fazer uma amostra das distribuições anteriores como esta:

In [57]:
n_socks = prior_n_socks.rvs()
prop_pairs = prior_prop_pair.rvs()

n_socks, prop_pairs

E utilizar os valores para calcular `n_pairs' e `n_odds':

In [58]:
n_pairs = np.round(n_socks//2 * prop_pairs)
n_odds = n_socks - n_pairs*2

n_pairs, n_odds

Agora, é a partir daí.

In [59]:
# Solution goes here

In [60]:
# Solution goes here

In [61]:
# Solution goes here

In [62]:
# Solution goes here

In [63]:
# Solution goes here

In [64]:
# Solution goes here

In [65]:
# Solution goes here

In [66]:
# Solution goes here

In [67]:
# Solution goes here

In [68]:
# Solution goes here

In [69]:
# Solution goes here